<a href="https://colab.research.google.com/github/AlbertoMontanelli/Machine-Learning/blob/class_unit/class_unit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Class: unit
The single unit $i$ is described by the following attributes and methods:


*   Input array -> $x_j$
*   Weight array -> $w_{i,j}$
*   Bias of the unit -> $b_i$
*   Output value -> $f(\sum_{j=0}^nw_{i,j}x_j+b_i)$ where f is the activation function, n the number of inputs



In [1]:
import numpy as np


def sigmoid(input_array, weight_array, bias):
  return 1 / (1 + np.exp(-(np.dot(input_array, weight_array) + bias)))

def tanh(input_array, weight_array, bias):
  return np.tanh(np.dot(input_array, weight_array) + bias)

def softmax(input_array, weight_array, bias):
  return np.exp(np.dot(input_array, weight_array) + bias) / np.sum(np.exp(np.dot(input_array, weight_array) + bias))

def softplus(input_array, weight_array, bias):
  return np.log(1 + np.exp(np.dot(input_array, weight_array) + bias))

def linear(input_array, weight_array, bias):
  return np.dot(input_array, weight_array) + bias

def ReLU(input_array, weight_array, bias):
  return np.maximum(np.dot(input_array,weight_array) + bias, 0)

class Unit:
  def __init__(self, input_array, weight_array, bias, activation_function): #costruttore della classe, qua definisco gli attributi della classe
    self.input_array = input_array
    self.weight_array = weight_array
    self.bias = bias
    self.activation_function = activation_function
    self.output_value = activation_function(input_array, weight_array, bias)


## Test for unit class:


*   Verification of its working by a simple test with a monodimensional array of inputs, weights e bias
*   Verification with multidimensional array and matrix products:
 *  input: $l$ (examples) x $n$ (features) -> we get a weight for each feature (it's indipendent by the number of patterns $l$). Then for the pattern $i$ we get $f(XW+b)=f(\sum_{j=0}^nx_{i,j}w_j+b)$.
 *$k$ units: matrix $X = l$ x $n$, matrix $W$ = $n$ x $k$. Then we get a matrix output $A$ = $l$ x $k$. Each value in the matrix is the output of the activation function for the unit $k$ respect to the example $i$ (many columns as the number of the units, many rows as the number of the examples)



In [2]:
# monodimensional array
x=np.array([1,2,3])
w=np.array([4,5,6])
b=1
neuron = Unit(x, w, b, ReLU)
output=neuron.output_value
print(output)

# multidimensional array
x=np.ones((5,10)) #5 examples, 10 features
w=np.ones((10,6)) #10 features, 6 units
print(x)
print(w)
b=1
neuron = Unit(x, w, b, softplus)
output=neuron.output_value
print(output) #we get 5x6 matrix

33
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
[[1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]]
[[11.0000167 11.0000167 11.0000167 11.0000167 11.0000167 11.0000167]
 [11.0000167 11.0000167 11.0000167 11.0000167 11.0000167 11.0000167]
 [11.0000167 11.0000167 11.0000167 11.0000167 11.0000167 11.0000167]
 [11.0000167 11.0000167 11.0000167 11.0000167 11.0000167 11.0000167]
 [11.0000167 11.0000167 11.0000167 11.0000167 11.0000167 11.0000167]]
